In [2]:
# Required import
!pip install -q confluent-kafka
print("Installed")

Installed


8. In Jupyter, write a program to subscribe to the homepage topic generated by the stream/table in Question 4 and display the messages to the console. 

In [1]:
import json
import pyspark
from pyspark.sql import SparkSession
from confluent_kafka import Consumer

In [3]:
spark = SparkSession.builder \
    .master("local") \
    .appName("jupyter-pyspark") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print("done")

done


In [7]:
consumer = Consumer({'bootstrap.servers' : 'broker:29092', 'group.id' : '*'})
print("TOPICS")
topics = consumer.list_topics() # this is a python dictionary
for topic in topics.topics.keys():
    if not topic.startswith("_"): # only show non-internal topics
        print(topic)


TOPICS
USER_ACTIVITY
default_ksql_processing_log
docker-connect-status
weblogs
docker-connect-offsets
docker-connect-configs
HOMEPAGE
atm


In [8]:
consumer.subscribe(["weblogs"])
maxcount = 10
count = 0
try:
    while True:
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        raw = msg.value().decode('utf-8')
        payload = json.loads(raw)
        count = count + 1
        print(f"Received message: {payload}")
        print(count)
        if count == maxcount:
            print("We could write this payload as a file to HDFS")
            count = 0
except KeyboardInterrupt:
    consumer.close()    

Received message: {'Uri': '/services', 'User': 'walt', 'TimeStamp': 1679165615000, 'Browser': 'edge', 'OS': 'win'}
1
Received message: {'Uri': '/contact', 'User': 'patty', 'TimeStamp': 1679165617000, 'Browser': 'firefox', 'OS': 'win'}
2
Received message: {'Uri': '/', 'User': 'mike', 'TimeStamp': 1679165620000, 'Browser': 'firefox', 'OS': 'osx'}
3
Received message: {'Uri': '/about', 'User': 'fred', 'TimeStamp': 1679165630000, 'Browser': 'safari', 'OS': 'osx'}
4
Received message: {'Uri': '/', 'User': 'fred', 'TimeStamp': 1679165636000, 'Browser': 'safari', 'OS': 'osx'}
5
Received message: {'Uri': '/', 'User': 'devin', 'TimeStamp': 1679165645000, 'Browser': 'edge', 'OS': 'win'}
6
Received message: {'Uri': '/', 'User': 'devin', 'TimeStamp': 1679165650000, 'Browser': 'edge', 'OS': 'win'}
7
Received message: {'Uri': '/', 'User': 'patty', 'TimeStamp': 1679165661000, 'Browser': 'firefox', 'OS': 'win'}
8
Received message: {'Uri': '/services', 'User': 'gigi', 'TimeStamp': 1679165663000, 'Browser